In [4]:
import streamlit as st
import plotly as px
import seaborn as sns
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openpyxl as op

In [93]:
status = pd.read_excel (r'E:/Administration/Life.xlsx', sheet_name='Status')
expenses = pd.read_excel (r'E:/Administration/Life.xlsx', sheet_name='Expenses')

e:\dev\py\personalfinance\venv\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
e:\dev\py\personalfinance\venv\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
e:\dev\py\personalfinance\venv\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Sparkline Group extension is not supported and will be removed
  warn(msg)


In [ ]:
#Incomes
Pre-Tax Salary
Salary
Pension
Bonus
House Rent


In [ ]:
#Expenses
Mortgage
Building Insurance
Contents Insurance
Landlord Insurance
Rates
Water
Health Insurance
Rent
Telstra
Power
Gym
Haircuts
Living (Food/Entertainment)
60k Car Loan
Car Insurance
Car Rego
Fuel (Calculator)
Servicing (Calculator)
Tyres (Calculator)
Dobell Rates
Dobell Water
University
Rental Management Fee
Rental Annual Fee
Rental Admin Fee


In [94]:
#Assets
HOUSEVALUE = 600000
CARVALUE = 65400

In [ ]:
#
Interest on Loan
Capital Works
Capital Allowances
University Car
Business Deductions

In [95]:
status = status[['Date', 'Account 1', 'Account 2',
       'Account 3', 'Account 4', 'Crypto', 'Shares Account', 'Shares',
       'Car Loan', 'Home Loan']]
status = status[status['Date'] < '2022-07-06']
status = status[status['Date'] > '2020-09-22']
status = status.reset_index(drop=True)

In [96]:
expenses = expenses[['Date', 'Invest Prop.', 'Shopping', 'Home', 'Groceries',
       'Transport', 'Utilities', 'Health', 'Eating Out', 'Entertainment',
       'Cash', 'Travel', 'Education', 'Fees & Interest']]       
expenses=expenses[14:]
expenses = expenses.reset_index(drop=True)

In [97]:
status['Equity'] = status['Account 1'] + status['Account 2'] + status['Account 3'] + status['Account 4'] + status['Crypto'] + status['Shares Account'] + status['Shares']
status['Debt'] = status['Car Loan'] + status['Home Loan']
status['Net Worth'] = status['Equity'] - status['Debt'] + HOUSEVALUE + CARVALUE

In [113]:
networthrate = (status['Net Worth'][int(len(status.index)-1)] - status['Net Worth'][0])/len(status.index)

In [112]:
savingrate

899.2488070393358